In [1]:
import pandas as pd
from utils_analysis import prepare_scores, plot_correlation
import matplotlib.pyplot as plt
import numpy as np

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/mila/f/floresl/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/mila/f/floresl/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/mila/f/floresl/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
df = pd.read_csv("data/squad/test.csv")

# "results/bart/r1/bart-base_data_squad_checkpoint-26280_squad.json",
# "results/bart/r2/bart-base_data_squad_checkpoint-26280_squad.json",
# "results/bart/r3/bart-base_data_squad_checkpoint-26280_squad.json",
# "results/t5/r1/flan-t5-base_data_squad_checkpoint-26280_squad.json",
# "results/t5/r2/flan-t5-base_data_squad_checkpoint-26280_squad.json",
# "results/t5/r3/flan-t5-base_data_squad_checkpoint-26280_squad.json",

df_scores, beam_search_results = prepare_scores(
    "results/t5/r3/flan-t5-base_data_squad_checkpoint-26280_squad.json",
    df["target"],
    "f1",
)

/home/mila/f/floresl/beam-search/utils_analysis.py:37: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  abs(spearmanr(ground_truth_score, confidence_score_dict[str(k)]).statistic)  # type: ignore


In [3]:
df_scores.drop(["sentences", "dropout_sentences"], axis=1).corr(method="spearman")["f1"]

length_normalized_log_probs    0.444738
mean_token_entropy             0.007599
dropout_bleu_variance         -0.282156
dropout_meteor_score           0.221183
dropout_entropy               -0.252736
dropout_disagreement          -0.389573
f1                             1.000000
beam_score_ratios_1            0.409458
beam_score_log_probs_1        -0.241033
beam_score_top_k_1             0.072919
beam_score_impt_wt_1          -0.149944
Name: f1, dtype: float64

In [4]:
beam_score_log_probs = pd.DataFrame(beam_search_results["beam_score_log_probs"])
worst_rouge_idxs = df_scores.sort_values(by="f1").index[:100]
best_rouge_idxs = df_scores.sort_values(by="f1").index[::-1][:100]

In [5]:
from utils_tail_probs import softmax, tail_index
from scipy.stats import spearmanr
from scipy.spatial.distance import jensenshannon

In [14]:
tail_indices = []
for i in range(len(beam_score_log_probs)):
    probs = softmax(beam_score_log_probs.iloc[i].to_numpy(), temperature=0.005)
    tail_indices.append(tail_index(probs))

spearmanr(tail_indices, df_scores["f1"])

SignificanceResult(statistic=0.42716153394871137, pvalue=1.2879423468677025e-45)

In [16]:
import json

baseline_probs = json.load(
    open(
        "/home/mila/f/floresl/beam-search/results/t5_baseline/google_flan-t5-base_hotpotqa.json",
        "r",
    )
)

In [17]:
baseline_log_probs = baseline_probs["beam_score_log_probs"]
baseline_log_probs = pd.DataFrame(baseline_log_probs).values

beam_score_log_probs = beam_score_log_probs.values

In [21]:
js_distances = []
for i in range(1000):
    js = jensenshannon(
        softmax(beam_score_log_probs[i], temperature=0.05),
        softmax(baseline_log_probs[i], temperature=0.05),
    )
    js_distances.append(js)
spearmanr(js_distances, df_scores["f1"])

SignificanceResult(statistic=0.3346126509540507, pvalue=1.3829398691268918e-27)

In [32]:
js_distances = []
for i in range(1000):
    js = jensenshannon(
        softmax(beam_score_log_probs[i], temperature=0.05), np.array([1 / 100] * 100)
    )
    js_distances.append(js)
spearmanr(js_distances, df_scores["f1"])

SignificanceResult(statistic=0.3494262268209138, pvalue=4.317629104247729e-30)